In [ ]:
#LƯU Ý. ĐỂ CHẠY ĐƯỢC OFFLINE, PHẢI CHẠY LẦN ĐẦU ONLINE TRƯỚC (ĐỂ CODE TẢI CÁC MODEL VỀ CACHE)

In [ ]:
!rmdir /s /q "../Whisper/content/audio_split_files"

In [ ]:
!mkdir "../Whisper/content"
!mkdir "../Whisper/content/audio_split_files"

A subdirectory or file ../Whisper/content already exists.


In [ ]:
import subprocess
import os
#Chép file .mp3 gốc vào thư mục Document/Whisper/content
filename = "1hour_qh.mp3"
#filename = "8min.mp3"

#Xác định thời gian cắt nhỏ, càng cắt nhỏ càng ít bị lỗi, nhưng dịch sẽ ko tốt
#segment_time = 600 #10 phút
#segment_time = 300 #5 phút
segment_time = 180 #3 phút

ffmpeg_command = f'ffmpeg -i "../Whisper/content/{filename}" -f segment -segment_time {segment_time} -c copy "../Whisper/content/audio_split_files/{filename}_%03d.mp3"'
subprocess.run(ffmpeg_command, shell=True)

CompletedProcess(args='ffmpeg -i "../Whisper/content/1hour_qh.mp3" -f segment -segment_time 180 -c copy "../Whisper/content/audio_split_files/1hour_qh.mp3_%03d.mp3"', returncode=0)

In [ ]:
import whisper


In [ ]:
import torch
import torchaudio
SAMPLING_RATE = 16000

In [ ]:
import os

# Lấy đường dẫn thư mục người dùng hiện tại
user_folder = os.path.expanduser("~")
# Đường dẫn tới file lưu model và utils
data_vad_path = os.path.join(user_folder, '.cache', 'torch', 'hub', 'snakers4_silero-vad_master')
print("Đường dẫn đầy đủ: ", data_vad_path)

# Kiểm tra xem file đã tồn tại chưa
if os.path.exists(data_vad_path):
    # Nếu file tồn tại, load model và utils từ file
    print("Đã có cache, load từ cache")
    model_vad, utils = torch.hub.load(repo_or_dir='snakers4/silero-vad',
                                  model='silero_vad',
                                  force_reload=False,
                                  onnx=False)
else:
    print("Chưa có cache, cần kết nối Internet lần đầu")
    model_vad, utils = torch.hub.load(repo_or_dir='snakers4/silero-vad',
                                  model='silero_vad',
                                  force_reload=True,
                                  onnx=False)

(get_speech_timestamps,
     save_audio,
     read_audio,
     VADIterator,
     collect_chunks) = utils

Đường dẫn đầy đủ:  C:\Users\nhquan\.cache\torch\hub\snakers4_silero-vad_master
Đã có cache, load từ cache


Using cache found in C:\Users\nhquan/.cache\torch\hub\snakers4_silero-vad_master


In [ ]:
def has_duplicates(text):
  # Tách text thành các đoạn 30 từ
  words = text.split()
  for i in range(0, len(words), 40):
    if i > len(words) - 40: #Đã chạy đến cuối
      last5word = ' '.join(words[-6:])
      if text.count(last5word) >= 4:
        return True
      return False
    first3words = ' '.join(words[i:i+3])
    subtext = ' '.join(words[i:i+40])
    if subtext.count(first3words) >= 3:
      return True

In [ ]:
# Đường dẫn đến thư mục chứa tất cả các tệp âm thanh
audio_directory = "../Whisper/content/audio_split_files/"

# Danh sách các tệp âm thanh trong thư mục
audio_split_files = os.listdir(audio_directory)

# Load mô hình Whisper
#model = whisper.load_model("large-v3")
#model = whisper.load_model("medium")

try:
    test_init = model
except NameError:
    model = whisper.load_model("medium")

In [ ]:
print("Torch version:", torch.__version__)
print("Torchaudio version:", torchaudio.__version__)

Torch version: 2.1.1+cu118
Torchaudio version: 2.1.1+cu118


In [ ]:
import time

total_time_start = time.time()
start_time = time.time()

# Khởi tạo chuỗi để lưu kết quả
all_transcriptions = ""

# Lặp qua từng tệp âm thanh và thực hiện xử lý
for audio_file in sorted(audio_split_files):
    if audio_file.endswith(".mp3"):
        audio_path = os.path.join(audio_directory, audio_file)
        result = model.transcribe(audio_path)
        transcription_text = result['text']
        if has_duplicates(transcription_text) == True: #Kiểm tra lần 1 - condition_on_previous_text
            print("Bị duplicated lần 1:", audio_path)
            result = model.transcribe(audio_path, condition_on_previous_text=False)
            transcription_text = result['text']
            if has_duplicates(transcription_text) == True: #Kiểm tra lần 2 - VAD
                #VAD audio file
                print("Bị duplicated lần 2:", audio_path)
                wav = read_audio(audio_path, sampling_rate=SAMPLING_RATE)
                # get speech timestamps from full audio file
                speech_timestamps = get_speech_timestamps(wav, model_vad, sampling_rate=SAMPLING_RATE, threshold = 0.6)
                # merge all speech chunks to one audio
                save_audio('../Whisper/content/audio_split_files/temp.wav', collect_chunks(speech_timestamps, wav), sampling_rate=SAMPLING_RATE)
                result = model.transcribe('../Whisper/content/audio_split_files/temp.wav', condition_on_previous_text=False)
                transcription_text = result['text']
                if has_duplicates(transcription_text) == True: #Kiểm tra lần 3
                    print("Bị duplicated lần 3 - Chấp nhận:", audio_path)
        all_transcriptions += f"File: {audio_file}, Transcription: {transcription_text}\n"

# Ghi kết quả vào tệp
with open("../Whisper/content/all_transcriptions.txt", "w", encoding="utf-8") as output_file:
    output_file.write(all_transcriptions)

end_time = time.time()
# In thông báo khi hoàn thành
print("Tất cả kết quả đã được ghi vào tệp 'all_transcriptions.txt' sau %.2f (s)" % (end_time - start_time))


Bị duplicated lần 1: ../Whisper/content/audio_split_files/1hour_qh.mp3_016.mp3
Tất cả kết quả đã được ghi vào tệp 'all_transcriptions.txt' sau 894.46 (s)


In [ ]:
# Sử dụng lệnh cat để hiển thị nội dung tệp "all_transcriptions.txt"
print(all_transcriptions)